In [ ]:
# !pip uninstall opencv-python
# !pip uninstall opencv-contrib-python
# !pip3 install opencv-contrib-python
import tensorflow as tf
from matplotlib.pyplot import imshow
import numpy as np
import cv2
from cv2 import dnn_superres
# from tensorflow import keras
from keras.preprocessing.image import img_to_array
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
from tqdm import tqdm  # for progress bar
import os
import sys
import seaborn as sns

#print info regarding GPU
print(tf.__version__)
# gpus = tf.config.list_physical_devices('GPU')
# tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)])
# test_path = '/content/drive/MyDrive/data/data/archive (1)/custom_test2/'

# test_path = '/content/drive/MyDrive/data/data/archive (1)/custom_test/new_noisy_test/'
test_path = '/content/drive/MyDrive/data/data/archive (1)/custom_test/clean_test/'

testing_data = os.listdir(test_path)

noise_factor = 0.05
np.random.seed(42)
img_dimension = 150
test_img_data = []
train_img_data = []

showloss = []
no_of_epochs = 0

for i in tqdm(testing_data):
    img1 = cv2.imread(test_path + i, 1)  # Change 1 to 0 for grey images
    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)  # Changing BGR to RGB to show images in true colors
    img1 = cv2.resize(img1, (img_dimension, img_dimension))
    test_img_data.append(img_to_array(img1))

test_img_array = np.reshape(test_img_data, (len(test_img_data), img_dimension, img_dimension, 3))
test_img_array = test_img_array.astype('float32') / 255.

# adding some noise

x_test_noisy = test_img_array + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=test_img_array.shape)




## conv model

def get_model():
    generate_model = Sequential()
    generate_model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(img_dimension, img_dimension, 3)))
    generate_model.add(MaxPooling2D((2, 2), padding='same'))

    generate_model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    generate_model.add(UpSampling2D((2, 2)))

    generate_model.add(Conv2D(3, (3, 3), activation='relu', padding='same'))

    generate_model.compile(optimizer='adam', loss='mean_squared_error')

    generate_model.summary()
    return generate_model




## dense model

# def get_model():
#     generate_model = Sequential()
#     generate_model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(img_dimension, img_dimension, 3)))
#     generate_model.add(tf.keras.layers.Dense(32, activation='relu'))
#     generate_model.add(MaxPooling2D((2, 2), padding='same'))
#     generate_model.add(tf.keras.layers.Dense(32, activation='relu'))
#     generate_model.add(tf.keras.layers.Dense(32, activation='relu'))
#     # # for i in range(2):
#     # generate_model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
#     # generate_model.add(MaxPooling2D((2, 2), padding='same'))
#     # #
#     # # for j in range(2):
#     # generate_model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
#     # generate_model.add(UpSampling2D((2, 2)))

#     # generate_model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
#     generate_model.add(UpSampling2D((2, 2)))
#     # generate_model.add(tf.keras.layers.Dense(32, activation='relu'))
#     generate_model.add(tf.keras.layers.Dense(32, activation='relu'))
#     generate_model.add(tf.keras.layers.Dense(32, activation='relu'))
#     # generate_model.add(tf.keras.layers.Dense(32, activation='relu'))
#     # generate_model.add(tf.keras.layers.Dense(32, activation='relu'))
#     # generate_model.add(tf.keras.layers.Dense(32, activation='relu'))
#     generate_model.add(Conv2D(3, (3, 3), activation='relu', padding='same'))
#     generate_model.compile(optimizer='adam', loss='mean_squared_error')

#     generate_model.summary()
#     return generate_model

f = open("/content/drive/MyDrive/data/data/data.txt", "r")
file_no = int(f.readline())
no_of_files = int(f.readline())
no_of_iterations = int(f.readline())
no_of_epochs = int(f.readline())
name_iter = int(f.readline())  # for saving iteration
f.close()

if no_of_iterations == 9 and file_no == 0:
    print("no old model found")
    model = get_model()
else:

    print("no of iter : ", no_of_iterations, " no of files : ", no_of_files)
    model = tf.keras.models.load_model("/content/drive/MyDrive/data/data/autoencoder.model")
    print("model.evaluate")
    model.evaluate(x_test_noisy, test_img_array)
    model.summary()



h=file_no
while h < no_of_files:
    no_of_epochs+=1
    train_img_data.clear()
    train_path = '/content/drive/MyDrive/data/data/archive (1)/custom_train/clean_train'+str(h)+'/'
    training_data = os.listdir(train_path)

    for i in tqdm(training_data):
        img = cv2.imread(train_path + i, 1)  # Change 1 to 0 for grey images
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Changing BGR to RGB to show images in true colors
        img = cv2.resize(img, (img_dimension, img_dimension))
        train_img_data.append(img_to_array(img))

    train_img_array = np.reshape(train_img_data, (len(train_img_data), img_dimension, img_dimension, 3))
    train_img_array = train_img_array.astype('float32') / 255.

    x_train_noisy = train_img_array + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=train_img_array.shape)

    for k in range(no_of_iterations):
        hist = model.fit(x_train_noisy, train_img_array, epochs=no_of_epochs, shuffle=True)

        denoised_img = model.predict(x_test_noisy)
        # imshow(x_test_noisy)
        plt.figure(figsize=(20, 8))  # size of window

        for i in range(1, 6):
            ax = plt.subplot(1, 5, i)  # subplot(no of rows of images, no of columns of images, position of image )

            plt.imshow((denoised_img[i] * 255).astype(np.uint8))
        plt.savefig("/content/drive/MyDrive/data/data/results/" + str(name_iter) + 'result.jpg')
        print(name_iter)
        name_iter += 1
        # no_of_files=h
        print("file no", str(h))
        print("no of iterations",str(no_of_iterations - k))
        print("no of epochs",str(no_of_epochs))
        f = open("/content/drive/MyDrive/data/data/data.txt", "w")
        f.write(str(h))
        f.write("\n")
        f.write(str(no_of_files))
        f.write("\n")
        f.write(str(no_of_iterations - k))
        f.write("\n")
        f.write(str(no_of_epochs))
        f.write("\n")
        f.write(str(name_iter))
        f.close()

        model.save('/content/drive/MyDrive/data/data/autoencoder.model')

        # plt.show()
    f.close()

    model.save('/content/drive/MyDrive/data/data/autoencoder.model')
    h += 1

print("model.evaluate")
model.evaluate(x_test_noisy, test_img_array)
denoised_img = model.predict(x_test_noisy)


# final comparison between original and end result
plt.figure(figsize=(20, 8))  # size of window
for i in range(1, 6):
    # display original
    ax = plt.subplot(2, 5, i)
    # plt.imshow(x_test_noisy[i].reshape(28, 28))
    plt.imshow((x_test_noisy[i] * 255).astype(np.uint8))

    # display reconstructed (after noise removed) image
    ax = plt.subplot(2, 5, 5 + i)
    plt.imshow((denoised_img[i] * 255).astype(np.uint8))

plt.savefig('/content/drive/MyDrive/data/data/results/Final_result.jpg')

for i in range(1, 6):
    plt.figure(figsize=(2, 2))  # size of window
    plt.imshow((denoised_img[i] * 255).astype(np.uint8))
    plt.axis('off')
    plt.savefig("/content/drive/MyDrive/data/data/results/upscaled_final_images/" + str(i) + '_upscaled_test.jpg',bbox_inches='tight',pad_inches = 0)

# importing upscaleing model

upScaleingModel = dnn_superres.DnnSuperResImpl_create()
upScaleingModel.readModel("/content/drive/MyDrive/data/data/upscale_model/saved_model.pb")
# Set CUDA backend and target to enable GPU inference
# upScaleingModel.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
# upScaleingModel.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
# Set the desired model and scale to get correct pre- and post-processing
upScaleingModel.setModel("edsr", 4)
# # Read image

for i in range(1, 6):
    image = cv2.imread("/content/drive/MyDrive/data/data/results/upscaled_final_images/" + str(i) + '_upscaled_test.jpg')
    # Upscale the image
    result = upScaleingModel.upsample(image)
    # Save the image
    cv2.imwrite("/content/drive/MyDrive/data/data/results/upscaled_final_images/" + str(i) + '_upscaled_result.jpg', result)

plt.figure(figsize=(10, 10))
plt.plot(hist.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.show()


sys.exit("here we go again")




2.8.0


100%|██████████| 35/35 [00:00<00:00, 186.79it/s]


no old model found
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 150, 150, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 75, 75, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 75, 75, 32)        9248      
                                                                 
 up_sampling2d (UpSampling2D  (None, 150, 150, 32)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 150, 150, 3)       867       
                                                                 
Total params: 11,011
Trainable params

100%|██████████| 500/500 [00:05<00:00, 91.76it/s] 


Epoch 1/21
16/16 [==============================] - 1s 45ms/step - loss: 0.1554
Epoch 2/21
16/16 [==============================] - 1s 38ms/step - loss: 0.0337
Epoch 3/21
16/16 [==============================] - 1s 38ms/step - loss: 0.0195
Epoch 4/21
16/16 [==============================] - 1s 37ms/step - loss: 0.0145
Epoch 5/21
16/16 [==============================] - 1s 37ms/step - loss: 0.0119
Epoch 6/21
16/16 [==============================] - 1s 37ms/step - loss: 0.0103
Epoch 7/21
16/16 [==============================] - 1s 38ms/step - loss: 0.0090
Epoch 8/21
16/16 [==============================] - 1s 38ms/step - loss: 0.0079
Epoch 9/21
16/16 [==============================] - 1s 38ms/step - loss: 0.0070
Epoch 10/21
16/16 [==============================] - 1s 37ms/step - loss: 0.0064
Epoch 11/21
16/16 [==============================] - 1s 37ms/step - loss: 0.0059
Epoch 12/21
16/16 [==============================] - 1s 38ms/step - loss: 0.0056
Epoch 13/21
16/16 [==================

...

66
file no 7
no of iterations 6
no of epochs 28
INFO:tensorflow:Assets written to: /content/drive/MyDrive/data/data/autoencoder.model/assets
Epoch 1/28
16/16 [==============================] - 1s 37ms/step - loss: 0.0020
Epoch 2/28
16/16 [==============================] - 1s 36ms/step - loss: 0.0019
Epoch 3/28
16/16 [==============================] - 1s 37ms/step - loss: 0.0019
Epoch 4/28
16/16 [==============================] - 1s 38ms/step - loss: 0.0019
Epoch 5/28
16/16 [==============================] - 1s 38ms/step - loss: 0.0020
Epoch 6/28
16/16 [==============================] - 1s 38ms/step - loss: 0.0019
Epoch 7/28
16/16 [==============================] - 1s 38ms/step - loss: 0.0019
Epoch 8/28
16/16 [==============================] - 1s 38ms/step - loss: 0.0019
Epoch 9/28
16/16 [==============================] - 1s 38ms/step - loss: 0.0020
Epoch 10/28
16/16 [==============================] - 1s 38ms/step - loss: 0.0019
Epoch 11/28
16/16 [==============================] - 1s 38

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:150: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


67
file no 7
no of iterations 5
no of epochs 28
INFO:tensorflow:Assets written to: /content/drive/MyDrive/data/data/autoencoder.model/assets
Epoch 1/28
16/16 [==============================] - 1s 43ms/step - loss: 0.0019
Epoch 2/28
16/16 [==============================] - 1s 38ms/step - loss: 0.0019
Epoch 3/28
16/16 [==============================] - 1s 39ms/step - loss: 0.0019
Epoch 4/28
16/16 [==============================] - 1s 38ms/step - loss: 0.0019
Epoch 5/28
16/16 [==============================] - 1s 38ms/step - loss: 0.0019
Epoch 6/28
16/16 [==============================] - 1s 38ms/step - loss: 0.0020
Epoch 7/28
16/16 [==============================] - 1s 38ms/step - loss: 0.0021
Epoch 8/28
16/16 [==============================] - 1s 38ms/step - loss: 0.0020
Epoch 9/28
16/16 [==============================] - 1s 39ms/step - loss: 0.0019
Epoch 10/28
16/16 [==============================] - 1s 38ms/step - loss: 0.0019
Epoch 11/28
16/16 [==============================] - 1s 39

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:150: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


68
file no 7
no of iterations 4
no of epochs 28
